In [ ]:
# %pip install llama-index-llms-ollama llama-index-readers-obsidian llama-index-llms-langchain llama-index-graph-stores-nebula
# !pip install llama-index-multi-modal-llms-ollama
# !pip install llama-index-readers-file
# !pip install unstructured
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-vector-stores-chroma
# !pip install llama_index pyvis IPython

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb

from llama_index.core import ServiceContext

from llama_index.llms.langchain import LangChainLLM
from llama_index.core import PromptTemplate

from llama_index.readers.obsidian import ObsidianReader

# from llama_index.multi_modal_llms.ollama import OllamaMultiModal
# from llama_index.core.llms import ChatMessage

# from llama_index.core.indices.multi_modal.base import (
#     MultiModalVectorStoreIndex,
# )
# from llama_index.core.program import MultiModalLLMCompletionProgram

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings


from IPython.display import Markdown, display

In [ ]:
llm = Ollama(base_url='http://tinybot:11435', model="llava:7b-v1.6-mistral-q4_0", request_timeout=360.0)

In [ ]:
chroma_client = chromadb.HttpClient(host='http://192.168.36.21:8000')
chroma_collection = chroma_client.get_or_create_collection("obsidian10")

# # define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
!pip install vectorflow_client

In [ ]:
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

from pathlib import Path
from typing import TYPE_CHECKING, Any, List

if TYPE_CHECKING:
    from langchain.docstore.document import Document as LCDocument

from llama_index.core.readers.base import BaseReader
from llama_index.readers.file import MarkdownReader
from llama_index.core.schema import Document

# loader = PyMuPDFReader()
# documents = loader.load(file_path="Library/Docs/papers/nlp/Retrieval-Augmented Generation for Knowledge Intensive NLP Tasks.pdf")

documents = ObsidianReader(
    "/home/jovyan/Workspace/syncopatedNotes/website/_notes"
).load_data()

# documents = [
#   doc.to_langchain_format()
#   for doc in documents
# ]

# documents = [
#   doc.to_embedchain_format()
#   for doc in documents
# ]

#print(documents)


In [ ]:
from llama_index.core import SimpleDirectoryReader

# only load markdown files
required_exts = [".md"]

reader = SimpleDirectoryReader(
    input_dir="/home/jovyan/Workspace/syncopatedNotes/website/",
    required_exts=required_exts,
    recursive=True,
    filename_as_id=True,
)

# all_docs = []
# for docs in reader.iter_data():
#     for doc in docs:
#         # do something with the doc
#         print(doc.text)
#         doc.text = doc.text.upper()
#         all_docs.append(doc)

# print(len(all_docs))


documents = reader.load_data(num_workers=10)


In [ ]:
# import spacy

# nlp = spacy.load("en_core_web_sm")
# doc = nlp(documents[4].text)
# for sent in doc.sents:
#     print(sent.text)


In [ ]:

# nlp = spacy.load("en_core_web_sm", exclude=["parser"])
# nlp.enable_pipe("senter")
# doc = nlp(documents[4].text)
# for sent in doc.sents:
#     print(sent.text)

# import spacy
# from spacy.lang.en import English

# nlp = English()  # just the language with no pipeline
# nlp.add_pipe("sentencizer")
# doc = nlp(documents[4].text)
# for sent in doc.sents:
#     print(sent.text)


In [ ]:
text_parser = SentenceSplitter(
    chunk_size=400,
    separator=". ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
print(text_chunks[4])

In [ ]:
# index = MultiModalVectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     embed_model=embed_model,
# )
# qa_tmpl_str = (
#     "Context information is below.\n"
#     "---------------------\n"
#     "{context_str}\n"
#     "---------------------\n"
#     "Given the context information and not prior knowledge, "
#     "answer the query.\n"
#     "Query: {query_str}\n"
#     "Answer: "
# )
# qa_tmpl = PromptTemplate(qa_tmpl_str)

# query_engine = index.as_query_engine(
#     multi_modal_llm=mm_model, text_qa_template=qa_tmpl
# )

In [ ]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding
    
vector_store.add(nodes)

In [ ]:

query_str = "Describe RAG process"
query_embedding = embed_model.get_query_embedding(query_str)

from llama_index.core.vector_stores import VectorStoreQuery

# query_mode = "default"
# query_mode = "sparse"
query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=40, mode=query_mode
)


In [ ]:
query_result = vector_store.query(vector_store_query)

from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))
    
#print(nodes_with_scores)
print(query_result.nodes[0].get_content())


In [ ]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a ChromaVectorStore"""

    def __init__(
        self,
        vector_store: ChromaVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores
    
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=40
)


from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

query_str = "Describe RAG process"

response = query_engine.query(query_str)
print(str(response))